In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas_ta as ta
from tensorflow.keras.losses import Huber

# List of ticker symbols (example of 50 different stocks)
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'BRK.B', 'JNJ', 'V', 'WMT',
    'JPM', 'PG', 'MA', 'DIS', 'NVDA', 'HD', 'PYPL', 'BAC', 'VZ', 'ADBE',
    'NFLX', 'INTC', 'CMCSA', 'PFE', 'KO', 'PEP', 'T', 'CSCO', 'MRK', 'ABT',
    'XOM', 'CVX', 'NKE', 'CRM', 'MDT', 'MCD', 'UNH', 'WFC', 'BMY', 'ACN',
    'TXN', 'AMGN', 'COST', 'QCOM', 'NEE', 'LLY', 'TMO', 'IBM', 'HON', 'GE'
]

# Adjust the date range to the last 60 days
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=59)).strftime('%Y-%m-%d')

In [ ]:


# Function to download and preprocess data for a given ticker
def preprocess_data(ticker):
    data = yf.download(ticker, start=start_date, end=end_date, interval='5m')
    data.index.name = 'Date'
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['RSI'] = ta.rsi(data['Close'], window=14)
    data.dropna(inplace=True)
    return data

# Combine all stocks' data into a single DataFrame
combined_data = pd.DataFrame()

for ticker in tickers:
    stock_data = preprocess_data(ticker)
    stock_data['Ticker'] = ticker  # Add ticker column to identify the stock
    combined_data = pd.concat([combined_data, stock_data])

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data.drop(columns=['Ticker']))

# Function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length, 3]  # Predicting the 'Close' price
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Parameters
SEQ_LENGTH = 200

# Create sequences
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Split the data into training and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(SEQ_LENGTH, X.shape[2])))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

# model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(optimizer='adam', loss=Huber())

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions and the actual values
predicted_close = scaler.inverse_transform(
    np.concatenate((X_test[:, -1, :-1], predictions), axis=1))[:, -1]
actual_close = scaler.inverse_transform(
    np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

# Plot the predictions
plt.figure(figsize=(14, 5))
plt.plot(combined_data.index[-len(actual_close):], actual_close, color='blue', label='Actual')
plt.plot(combined_data.index[-len(predicted_close):], predicted_close, color='red', label='Predicted')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()



In [162]:
a = model.predict(X_test[15:16])
print(a, y_test[15])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[0.18597367]] 0.18600277235129622


In [ ]:
# Trading strategy simulation
initial_balance = 10000  # Initial amount of moneyq
balance = initial_balance
positions = 0
trades = []
stop_loss_pct = 0.02  # Stop loss percentage
take_profit_pct = 0.04  # Take profit percentage

for i in range(1, len(predicted_close)):
    if predicted_close[i] > actual_close[i-1] and positions == 0:  # Buy signal
        positions = balance / actual_close[i]  # Buy as many as possible
        buy_price = actual_close[i]
        balance = 0  # All money is now in stock
        trades.append((combined_data.index[-len(predicted_close)+i], 'buy', actual_close[i]))
    elif positions > 0:  # Evaluate selling conditions
        sell_price = actual_close[i]
        if (sell_price >= buy_price * (1 + take_profit_pct)) or (sell_price <= buy_price * (1 - stop_loss_pct)) or (predicted_close[i] < actual_close[i-1]):
            balance = positions * actual_close[i]  # Sell all positions
            positions = 0  # No more stock holdings
            trades.append((combined_data.index[-len(predicted_close)+i], 'sell', actual_close[i]))

# Final balance calculation
if positions > 0:  # If holding positions, sell them at the last close price
    balance = positions * actual_close[-1]
    positions = 0

profit = balance - initial_balance

# Print trading results
print(f"Initial balance: ${initial_balance:.2f}")
print(f"Final balance: ${balance:.2f}")
print(f"Total profit: ${profit:.2f}")

# Analyze the trades
trade_df = pd.DataFrame(trades, columns=['Date', 'Action', 'Price'])

plt.figure(figsize=(14, 5))
plt.plot(combined_data.index[-len(actual_close):], actual_close, color='blue', label='Actual Price')
buy_signals = trade_df[trade_df['Action'] == 'buy']
sell_signals = trade_df[trade_df['Action'] == 'sell']
plt.scatter(buy_signals['Date'], buy_signals['Price'], marker='^', color='green', label='Buy Signal', alpha=1)
plt.scatter(sell_signals['Date'], sell_signals['Price'], marker='v', color='red', label='Sell Signal', alpha=1)
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.title('Trading Strategy based on LSTM Predictions with Stop-Loss and Take-Profit')
plt.show()